In [2]:
!nvidia-smi

Wed May 12 12:03:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   40C    P0    29W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import os
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor) # 앞의 2개 --> mmdet/datasets/builder.py에 존재 / 마지막 1개 --> 
from mmdet.models import build_detector #mmdet/models/builder.py에 존재
from mmdet.apis import (train_detector, single_gpu_test, init_detector, inference_detector, show_result_pyplot) 
    # train_detector --> mmdet/apis/train.py에 존재, init_detector, inference_detector, show_result_pyplot --> mmdet/apis/inference.py에 존재, single_gpu_test -->  mmdet/apis/test.py에 존재
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np
import json, wandb

## train part

In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
#cfg = Config.fromfile('../configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')## faster_rcnn_hrnetv2p_w40_2x_coco
#파일 교체 시 여기 이름 바꾸기
config_dir_name = 'faster_rcnn'
config_file_name = 'faster_rcnn_r50_fpn_1x_coco'
cfg = Config.fromfile('../configs/'+config_dir_name+'/'+config_file_name+'.py')

PREFIX = '/opt/ml/input/data/'
#working directory 이름 설정
#sub_working_dir_name = 'faster_rcnn_r50_fpn_1x_trash'
working_dir_name = '../work_dirs/' + config_file_name

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 16    #batch size

cfg.seed=42 #42로 고정
cfg.gpu_ids = [0]
cfg.work_dir = working_dir_name

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

#wandb setting


## wandb setting

In [5]:
group_name = 'hrnet'; project_name = 'gjtak'; run_name = config_file_name
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
cfg.log_config.hooks[1].init_kwargs['group']=group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = config_file_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config

### config 저장

In [6]:
# config 객체 -> 파일(json)로 저장(저장 장소 = working directory와 같이)
#working directory 없으면 생성하기
try:
    if not os.path.exists(working_dir_name):
        os.makedirs(working_dir_name)
except OSError :
    print('Error: Creating directory :: ' + working_dir_name)

#cfg 객체 -> string(str이용) -> dict(eval이용) -> json 으로 저장
cfg_str = str(cfg)[str(cfg).find(')')+3:]
cfg_dict = eval(cfg_str)
with open(working_dir_name+'/config.json','w') as config_json_file :
    json.dump(cfg_dict, config_json_file, indent="\t")


In [7]:
#config 저장 확인
with open(working_dir_name+'/config.json', 'r') as f :
    json_data = json.load(f)
#json_data

In [8]:
model = build_detector(cfg.model)

2021-05-12 12:03:49,713 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 12:03:49,714 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 12:03:49,999 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [9]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.72s)
creating index...
index created!


In [10]:
#wandb watch
#wandb.watch(model, log='all')

In [11]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-12 12:03:59,123 - mmdet - INFO - Start running, host: root@7abc6b7bc976, work_dir: /opt/ml/p3-ims-obd-hansarang/gjtak_works/Object Detection/code/mmdetection_trash/work_dirs/faster_rcnn_r50_fpn_1x_coco
2021-05-12 12:03:59,124 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.30s)
creating index...
index created!
wandb: Currently logged in as: pstage3_det (use `wandb login --relogin` to force relogin)


020, memory: 6897, loss_rpn_cls: 0.0329, loss_rpn_bbox: 0.0415, loss_cls: 0.2596, acc: 91.6809, loss_bbox: 0.2970, loss: 0.6310, grad_norm: 1.5671
2021-05-12 12:25:47,563 - mmdet - INFO - Epoch [9][140/164]	lr: 2.000e-03, eta: 0:06:07, time: 0.707, data_time: 0.018, memory: 6897, loss_rpn_cls: 0.0337, loss_rpn_bbox: 0.0419, loss_cls: 0.2418, acc: 92.1826, loss_bbox: 0.2739, loss: 0.5914, grad_norm: 1.4879
2021-05-12 12:25:54,589 - mmdet - INFO - Epoch [9][150/164]	lr: 2.000e-03, eta: 0:06:00, time: 0.703, data_time: 0.018, memory: 6897, loss_rpn_cls: 0.0347, loss_rpn_bbox: 0.0459, loss_cls: 0.2624, acc: 91.4319, loss_bbox: 0.3070, loss: 0.6500, grad_norm: 1.5084
2021-05-12 12:26:01,633 - mmdet - INFO - Epoch [9][160/164]	lr: 2.000e-03, eta: 0:05:53, time: 0.704, data_time: 0.018, memory: 6897, loss_rpn_cls: 0.0360, loss_rpn_bbox: 0.0454, loss_cls: 0.2670, acc: 91.5674, loss_bbox: 0.2988, loss: 0.6473, grad_norm: 1.5820
2021-05-12 12:26:04,524 - mmdet - INFO - Saving checkpoint at 9 epo

train/loss_rpn_cls,0.02873
train/loss_rpn_bbox,0.04247
train/loss_cls,0.24855
train/acc,91.88599
train/loss_bbox,0.29243
train/loss,0.61218
train/grad_norm,1.55846
learning_rate,0.0002
momentum,0.9
_runtime,1799
_timestamp,1620822838


train/loss_rpn_cls,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,▅█▇▇▅▆▇▅▃▅▄▃▄▅▄▄▃▄▄▃▄▄▄▃▃▃▃▃▂▁▂▃▂▂▁▁▂▂▁▂
train/loss_cls,▄██▇▇▆▇▆▅▆▆▅▅▆▅▅▅▄▅▄▄▅▅▃▄▄▃▄▃▂▂▃▂▃▁▁▂▂▁▂
train/acc,█▃▂▂▁▂▂▁▂▁▁▂▂▁▂▂▂▃▂▃▂▁▂▃▂▂▃▂▃▄▃▃▃▃▄▄▄▄▄▄
train/loss_bbox,▁▆▇██▇▇▇▇█▇▇▆▇▇▇▆▆▇▆▇▇▇▆▇▆▆▇▆▅▆▆▆▆▅▅▅▆▅▅
train/loss,▇█▇▇▇▆▆▆▅▆▅▅▄▅▄▄▄▃▅▃▄▄▄▃▄▃▃▃▃▁▂▃▂▂▁▁▂▂▁▂
train/grad_norm,▆▃▂▄█▄▃▃▃▃▅▃▂▃▃▂▂▃▅▂▂▃▃▂▃▃▃▁▁▁▁▁▂▂▁▁▂▁▁▂
learning_rate,▁▂▃▃▄▅▅▆▇██████████████████▂▂▂▂▂▂▂▂▂▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


## \[option\] check inference result picture

## Inference part

In [12]:
epoch = cfg.runner.max_epochs
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [13]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [14]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-12 12:34:04,620 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-12 12:34:04,621 - mmdet - INFO - Use load_from_torchvision loader
2021-05-12 12:34:04,849 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

Use load_from_local loader


In [ ]:
img = '/opt/ml/input/data/batch_01_vt/1259.jpg'
result = inference_detector(model, img)
# show the results
show_result_pyplot(model, img, result)

In [15]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 35.1 task/s, elapsed: 24s, ETA:     0s

In [16]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, 'submission_'+config_file_name+f'_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.7910051 166.29288 72.28441 258.1669 162.39...,batch_01_vt/0021.jpg
1,1 0.11672539 0.0 254.27222 278.9244 457.78162 ...,batch_01_vt/0028.jpg
2,1 0.5059585 102.46115 159.92076 377.29037 372....,batch_01_vt/0031.jpg
3,8 0.9910874 28.70813 229.26047 268.64853 373.9...,batch_01_vt/0032.jpg
4,1 0.37863806 379.50616 253.739 496.26514 351.7...,batch_01_vt/0070.jpg
